In [ ]:
import glob
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize
%matplotlib inline
plt.style.use('ggplot')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

In [ ]:
def read_images(parent_dir):
    images = np.vstack([np.asarray([imresize(imread(os.path.join(parent_dir,sdir,name)),(300,300))
                              for name in os.listdir(os.path.join(parent_dir,sdir))]) 
                                    for sdir in os.listdir(parent_dir)])
    n_subdir_files = np.asarray([len(os.listdir(os.path.join(parent_dir,sdir))) 
                                 for sdir in os.listdir(os.path.join(parent_dir))])
    labels = np.hstack([np.repeat(i,n_subdir_files[i],axis = 0) for i in range(len(n_subdir_files))])
    return images, labels

def save_dataset(images,labels):
    #NOTE: by default numpy add extension ".npy" to file name
    np.save("images",images)
    np.save("labels",labels)
    
def load_dataset(fimage,flabel):
    images = np.load(fimage)
    labels = np.load(flabel)
    return images, labels

def plot_random_images(images,labels,nimg = 4):
    sample_images = np.random.choice(labels,nimg)
    fig, asx = plt.subplots(nrows=1,ncols=nimg, figsize=(20,20),dpi = 800)
    for i in range(len(asx)):
        asx[i].imshow(images[sample_images[i]])
        asx[i].grid(False)
    plt.show()
    
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [ ]:
parent_dir = "Images"
images,labels = read_images(parent_dir)
plot_random_images(images,labels)
labels = one_hot_encode(labels)

#----Optionally save the numpy array to a file for later reuse----#
#save_dataset(images,labels)
#images, labels = load_dataset("images.npy","labels.npy")

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(1.0, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x,W, [1, 2, 2, 1], padding='SAME')

def apply_conv(x,kernel_size,num_channels,depth):
    weights = weight_variable([kernel_size, kernel_size, num_channels, depth])
    biases = bias_variable([depth])
    return tf.nn.relu(tf.add(conv2d(x, weights),biases))

def apply_max_pool(x,kernel_size,stride_size):
    return tf.nn.max_pool(x, ksize=[1, kernel_size, kernel_size, 1], 
                          strides=[1, stride_size, stride_size, 1], padding='SAME')

In [ ]:
rnd_indices = np.random.rand(len(labels)) < 0.70

train_x = images[rnd_indices]
train_y = labels[rnd_indices]
test_x = images[~rnd_indices]
test_y = labels[~rnd_indices]

In [ ]:
image_size = 300
num_labels = 2
num_channels = 3

batch_size = 5
kernel_size = 14
depth = 50
num_hidden = 500

learning_rate = 0.1
dropout = 0.9
training_epochs = 1000

In [ ]:
X = tf.placeholder(tf.float32, shape=[None,image_size,image_size,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])
#X_ = tf.reshape(X, [-1,image_size,image_size,num_channels])
keep_prob = tf.placeholder(tf.float32)

#------------------------------------------------------------------------------------#

c_1 = apply_conv(X,kernel_size,num_channels,depth)
p_1 = apply_max_pool(c_1,4,4)

c_2 = apply_conv(p_1,10,depth,depth)
p_2 = apply_max_pool(c_2,4,4)

c_3 = apply_conv(p_2,2,depth,depth)
c_4 = apply_conv(c_3,1,depth,depth)


#------------------------------------------------------------------------------------#

shape = c_4.get_shape().as_list()
c_4_flat = tf.reshape(c_4, [-1, shape[1] * shape[2] * shape[3]])

f_weights = weight_variable([shape[1] * shape[2] * depth, num_hidden])
f_biases = bias_variable([num_hidden])
f = tf.nn.sigmoid(tf.add(tf.matmul(c_4_flat, f_weights),f_biases))
#f = tf.nn.dropout(f, dropout)

out_weights = weight_variable([num_hidden, num_labels])
out_biases = bias_variable([num_labels])
y_ = tf.nn.softmax(tf.matmul(f, out_weights) + out_biases)

#------------------------------------------------------------------------------------#

In [ ]:
cross_entropy = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
cost_history = np.empty(shape=[1],dtype=float)

with tf.Session() as session:
    tf.initialize_all_variables().run()

    for epoch in range(training_epochs):    
        offset = (epoch * batch_size) % (train_y.shape[0] - batch_size)
        batch_x = train_x[offset:(offset + batch_size), :, :, :]
        batch_y = train_y[offset:(offset + batch_size), :]
        
        _, c = session.run([optimizer, cross_entropy],feed_dict={X: batch_x, Y : batch_y})
        cost_history = np.append(cost_history,c)
    
    print('Test accuracy: ',session.run(accuracy, feed_dict={X: test_x, Y: test_y}))
    
    fig = plt.figure(figsize=(15,10))
    plt.plot(cost_history)
    plt.axis([0,training_epochs,0,np.max(cost_history)])
    plt.show()